In [9]:
# test reading headers
import csv
with open('testdata/csv_test_in_001.csv', encoding = 'utf-8-sig', errors = 'ignore', newline = '') as file_in:
    reader = csv.reader(file_in)
    rowcount = 0
    rowsprocessed = 0
    headers = {}
    with open('csv_test_out_001.csv', mode = 'w') as file_out:
        writer = csv.writer(file_out, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        for row in reader:
            if rowcount == 0:
                for cell in row:
                    header = {row.index(cell): cell}
                    headers.update(header)
                rowcount += 1
            else:
                pass
print(headers)

{0: 'id', 1: 'string', 2: 'attribute', 3: 'value', 4: 'comment'}


In [10]:
# test writing new headers
import csv
with open('testdata/csv_test_in_001.csv', encoding = 'utf-8-sig', errors = 'ignore', newline = '') as file_in:
    reader = csv.reader(file_in)
    rowcount = 0
    rowsprocessed = 0
    headers = {}
    with open('../testdata/csv_test_out_001.csv', mode = 'w') as file_out:
        writer = csv.writer(file_out, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        newheaders = ['id', 'value', 'header']
        writer.writerow(newheaders)

In [9]:
import csv

with open('testdata/csv_test_in_001.csv', encoding = 'utf-8-sig', errors = 'ignore', newline = '') as file_in:
    reader = csv.reader(file_in)
    rowcount = 0
    emptyrows = 0
    headers = {}
    
    with open('testdata/csv_test_out_001.csv', encoding = 'utf-8-sig', mode = 'w', newline = '') as file_out:
        writer = csv.writer(file_out, delimiter = ',', quotechar = '"', quoting = csv.QUOTE_MINIMAL)
        newheaders = ['id', 'value', 'header']
        writer.writerow(newheaders)
        for row in reader:
            if rowcount == 0:
                for cell in row:
                    header = {row.index(cell): cell}
                    headers.update(header)
                rowcount += 1
            elif rowcount > 0 and row[0] != '':
                for cell in row:
                    newrow = []
                    if row.index(cell) == 0: # accessing the position of the cell in the row
                        pass
                    else:
                        newrow.append(row[0])
                        newrow.append(cell)
                        newrow.append(headers[row.index(cell)])
                        writer.writerow(newrow)
                rowcount += 1
            elif rowcount > 0 and row[0] == '':
                emptyrows += 1
            else:
                pass

# very minimal report here
print(f"{rowcount} rows containing data were processed")
if emptyrows == 1:
    print(f"{emptyrows} empty row was skipped")
elif emptyrows > 1:
    print(f"{emptyrows} empty rows were skipped")
else:
    pass

5 rows containing data were processed
1 empty row was skipped


# SOLVED
- Garbled characters
    - On first run there were garbled characters, but changing the `open()` encoding argument for the `csv.writer` object to match that of the `open()` encoding arg for the `csv.reader` object ('utf-8-sig') fixed it!
- Don't process blank rows
    - See `elif rowcount > 0 and row[0] != '':`
- Eliminate every-other-row blank in output file [1]; although this might not be necessary...
    - Here's something: [CSV file written with Python has blank lines between each row](https://stackoverflow.com/questions/3348460/csv-file-written-with-python-has-blank-lines-between-each-row)
    - Extremely easy fix: Add arg `newline = ''` to `open()` for file_out (to "suppress Windows line translation", per SO answer linked above) [3]

# WOULD LIKE TO DO
- Add some kind of testing of output file for accuracy


# NOTES
1. This doesn't show up in the [online rendering of the output CSV](https://gist.github.com/briesenberg07/1d3bdc9d079a8581768088dea1111f6b/cc7bca414690ff98ee0b24ffd09a38ffe2f55163#file-csv_test_out_001-csv), which is strange
2. Error when attempting to process while `csv_test_out_001` is open in Excel:

```
PermissionError: [Errno 13] Permission denied: '../testdata/csv_test_out_001.csv'
```
3. Per CSV library > [csv.writer](https://docs.python.org/3/library/csv.html#csv.writer):

> If csvfile is a file object, it should be opened with newline=''. [...] If newline='' is not specified, newlines embedded inside quoted fields will not be interpreted correctly, and on platforms that use \r\n linendings on write an extra \r will be added. It should always be safe to specify newline='', since the csv module does its own (universal) newline handling.